# Operating Experience Data Daily Update

This file performs the daily data updates for IOEB boards.  This includes the following:

1. COVID-19 Local Conditions Update from JHU CSSE on GitHub
2. Power Status Update from data warehouse and push to Box for INPO
3. Findings Data Update from data warehouse
4. Event Notification data update from data warehouse

Items to be added: 

1. Power Status push to Box for INPO
2. OpE Documents update from ADAMS public WBA and internal APIs

Requirements:

Python packages:
    pandas
    numpy
    datetime
    selenium
    os
    re
    shutil
    pyodbc
    time
    
Sharepoint folders synced to user's U.S. NRC OneDrive:
    
https://usnrc.sharepoint.com/teams/COVID-19SiteImpacts/Shared%20Documents/Forms/AllItems.aspx
    
https://usnrc.sharepoint.com/teams/ROPDashboards/Shared%20Documents/Forms/AllItems.aspx
    
    
Note:  to use selenium, you'll need to download chromedriver from:
  
https://sites.google.com/a/chromium.org/chromedriver/downloads
      
and copy it to the correct location:
    
C:\ProgramData\ChromeDriver\

Note:

If pyodbc fails to update from the data warehouse, attempt to connect to data warehouse via Excel prior to running code to establish connection.

In [1]:
# import all necessary python packages:

import pandas as pd
import numpy as np
import datetime
import os
import re
import shutil
import pyodbc
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import xml.etree.ElementTree as ET


In [2]:
# Set troubleshooting mode - this will display headers and other information if the user is troubleshooting
# Set to zero to suppress unecessary output for normal runs, set to 1 to view all output.
trouble_flag = 0;

## COVID-19 Data Update for Local Conditions (Site Status) Board

This portion of the script updates the source files from the COVID-19 Site Status Board.  It pulls the current data file from the JHU CSSE GitHub page at:

https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv

and performs calculations to determine the local condition surround NRC facilities, based on the EPZ counties surrounding each operating reactor and a list of counties of interest for other facilities.  The county list and other files are hosted on a Sharepoint Teams site at:

https://usnrc.sharepoint.com/teams/COVID-19SiteImpacts/Shared%20Documents/Forms/AllItems.aspx

This code assumes that the user has synced this folder to their U.S. NRC OneDrive

Author:  Rebecca Sigmon

In [3]:
# download current COVID-19 data from JHU CSSE:
url="https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
time_series = pd.read_csv(url)

# (Optional) - display header for the downloaded file
if trouble_flag == 1:
    time_series.head()


In [4]:
#create a combined_key to differentiate between counties with the same name in different states
time_series["Combined_Key"] = time_series["Admin2"]+", "+time_series["Province_State"]+", "+time_series["iso2"]

if trouble_flag == 1:
    time_series.head()

In [5]:
#grab the counties of interest from epz_counties spreadsheet
sites_old = pd.read_excel(os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_status\\epz_counties.xlsx')

if trouble_flag == 1:
    sites_old.head()

In [6]:
#grab the US population by county spreadsheet
pop = pd.read_excel(os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_status\\US_pop.xlsx')

if trouble_flag == 1:
    pop.head()

In [7]:
#join the site spreadsheet with the population spreadsheet to add a population column to the site spreadsheet
sites = pop.merge(sites_old, left_on = "Key", right_on = "Combined_Key", how = "inner")

if trouble_flag == 1:
    sites.head()

In [8]:
#make a dataframe with just the case data for the counties of interest
time_series_sites = time_series.merge(sites, left_on = "Combined_Key", right_on = "Combined_Key", how = "inner")
time_series_sites["County"] = time_series_sites["County_x"]
time_series_sites["State"] = time_series_sites["State_x"]

if trouble_flag == 1:
    time_series_sites.head()

In [9]:
#set up a dataframe with each county-site pair
counties_sites = time_series_sites[["FIPS", "Site", "Region", "County","State","Combined_Key", "Population"]]

if trouble_flag == 1:
    counties_sites.head()

In [10]:
# define the timeframe of interest in a way that doesn't have to be updated each day

startdate = datetime.datetime(2020,3,4)
today = datetime.datetime.today()
offset = 53 + (today-startdate).days

if trouble_flag == 1:
    offset

In [11]:
#extract just the confirmed case numbers by date
dates = time_series_sites.iloc[:,53:offset]

if trouble_flag == 1:
    dates.head()

In [12]:
# create time series dataframe:

county_time_series = pd.DataFrame(columns = ["FIPS","Site","Region","County","State","Combined_Key","Confirmed","Date",])

In [13]:
#take the data from the columns and make each county-date entry a new row
cols = dates.columns

for i in range(0,len(cols)):
    one_day = pd.DataFrame(columns = ["Confirmed", "Date"])
    one_day["Confirmed"] = dates.iloc[:,i]
    one_day["Date"] = cols[i]
    county_time = counties_sites.join(one_day, how = "left", sort = False)
    county_time_series = county_time_series.append(county_time)
#    county_time.append(one_day)
#    print(one_day)

if trouble_flag == 1:
    county_time_series.head()

C:\Users\JBC4\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [14]:
# reset time series index:

county_time_series.reset_index(inplace = True)
county_time_series.drop("index", axis = 1, inplace = True)

In [15]:
#convert date column to date-time object to allow future data manipulations
county_time_series["Date"] = pd.to_datetime(county_time_series.Date)

if trouble_flag == 1:
    county_time_series.tail()

In [16]:
#create a unique key other than the index to allow better defining relationships in powerBI
county_time_series["Unique"] = county_time_series["Site"] + ", "+ county_time_series["County"]+", "+county_time_series["Date"].astype(str)

if trouble_flag == 1:
    county_time_series.tail()

In [17]:
#sort by unique column then by date
county_time_series.sort_values(["Unique", "Date"], inplace = True)
county_time_series.reset_index(inplace = True)
county_time_series.drop("index", axis = 1, inplace = True)

if trouble_flag == 1:
    county_time_series.head()

In [18]:
#change names because of code merge
rolling_avg = county_time_series

if trouble_flag == 1:
    rolling_avg.head()

In [19]:
#calculate new cases each day by subracting previous day's confirmed cases 
rolling_avg["New_Cases"] = rolling_avg["Confirmed"]-rolling_avg["Confirmed"].shift(1)
rolling_avg = rolling_avg[rolling_avg["Date"] != "2020-03-04"]

if trouble_flag == 1:
    rolling_avg.tail()


In [20]:
#calculate the 14-day rolling average of confirmed cases using each of the previous 14 days' worth of data
rolling_avg["Rolling_Avg"] = (rolling_avg["Confirmed"]+rolling_avg["Confirmed"].shift(1)+rolling_avg["Confirmed"].shift(2)+rolling_avg["Confirmed"].shift(3)+rolling_avg["Confirmed"].shift(4)+rolling_avg["Confirmed"].shift(5)+rolling_avg["Confirmed"].shift(6)+rolling_avg["Confirmed"].shift(7)+rolling_avg["Confirmed"].shift(8)+rolling_avg["Confirmed"].shift(9)+rolling_avg["Confirmed"].shift(10)+rolling_avg["Confirmed"].shift(11)+rolling_avg["Confirmed"].shift(12)+rolling_avg["Confirmed"].shift(13))/14

if trouble_flag == 1:
    rolling_avg.head()

C:\Users\JBC4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
#calculate a 14-day rolling average of the number of new cases each day
rolling_avg["New_Case_Roll_Avg"] = (rolling_avg["New_Cases"]+rolling_avg["New_Cases"].shift(1)+rolling_avg["New_Cases"].shift(2)+rolling_avg["New_Cases"].shift(3)+rolling_avg["New_Cases"].shift(4)+rolling_avg["New_Cases"].shift(5)+rolling_avg["New_Cases"].shift(6)+rolling_avg["New_Cases"].shift(7)+rolling_avg["New_Cases"].shift(8)+rolling_avg["New_Cases"].shift(9)+rolling_avg["New_Cases"].shift(10)+rolling_avg["New_Cases"].shift(11)+rolling_avg["New_Cases"].shift(12)+rolling_avg["New_Cases"].shift(13))/14

if trouble_flag == 1:
    rolling_avg.head()

C:\Users\JBC4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
if trouble_flag == 1:
    rolling_avg.tail()

In [23]:
#calculate the change(slope) of the 14-day rolling average of new cases over assigned timeframe
timeframe = 14
rolling_avg["Change_New_Case_Avg"] = (rolling_avg["New_Case_Roll_Avg"] - rolling_avg["New_Case_Roll_Avg"].shift(timeframe-1))/timeframe

if trouble_flag == 1:
    rolling_avg.tail(timeframe)

C:\Users\JBC4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
offset = datetime.timedelta(days = 14)
firstdate = startdate+offset

if trouble_flag == 1:
    firstdate

In [25]:
rolling_avg = rolling_avg[rolling_avg["Date"] > firstdate]

if trouble_flag == 1:
    rolling_avg.head()

In [26]:
#normalize the change in the 14-day rolling average of new cases by dividing by the rolling avg of new cases
rolling_avg["Normalized_Change"] = rolling_avg.apply(lambda row: row.Change_New_Case_Avg/row.New_Case_Roll_Avg if row.New_Case_Roll_Avg != 0 else 0, axis = 1)

if trouble_flag == 1:
    rolling_avg.tail()

In [27]:
rolling_avg[["Confirmed","Population","New_Cases","Rolling_Avg","New_Case_Roll_Avg","Change_New_Case_Avg",
             "Normalized_Change"]] = rolling_avg[["Confirmed","Population","New_Cases","Rolling_Avg","New_Case_Roll_Avg",
                                                  "Change_New_Case_Avg","Normalized_Change"]].apply(pd.to_numeric)

In [28]:
if trouble_flag == 1:
    rolling_avg.dtypes

In [29]:
#define a dataframe that groups the counties for each site together
site_avg = rolling_avg.groupby(["Date","Site"])[["Confirmed","Population","New_Cases","Rolling_Avg","New_Case_Roll_Avg",
                                                 "Change_New_Case_Avg"]].sum().reset_index()

if trouble_flag == 1:
    site_avg.tail()

In [30]:
site_avg["Conf_Per_100k"] = site_avg.apply(lambda row: 100000*row.Confirmed/row.Population, axis = 1)
site_avg["New_Per_100k"]= site_avg.apply(lambda row: 100000*row.New_Cases/row.Population, axis = 1)
site_avg["Conf_Avg_Per_100k"] = site_avg.apply(lambda row: 100000*row.Rolling_Avg/row.Population, axis = 1)
site_avg["New_Avg_Per_100k"] = site_avg.apply(lambda row: 100000*row.New_Case_Roll_Avg/row.Population, axis = 1)


if trouble_flag == 1:
    site_avg.tail()

In [31]:
rolling_avg["Conf_Per_100k"] = rolling_avg.apply(lambda row: 100000*row.Confirmed/row.Population, axis = 1)
rolling_avg["New_Per_100k"]= rolling_avg.apply(lambda row: 100000*row.New_Cases/row.Population, axis = 1)
rolling_avg["Conf_Avg_Per_100k"] = rolling_avg.apply(lambda row: 100000*row.Rolling_Avg/row.Population, axis = 1)
rolling_avg["New_Avg_Per_100k"] = rolling_avg.apply(lambda row: 100000*row.New_Case_Roll_Avg/row.Population, axis = 1)


if trouble_flag == 1:
    rolling_avg.tail()

In [32]:
#define parameters for creating a separate dataframe with just current day's data
today = datetime.datetime.today()
offset2 = datetime.timedelta(days = 2)
olddate = today - offset2

if trouble_flag == 1:
    olddate

In [33]:
#keep only most current day's data
#rolling_avg["Date"] = pd.to_datetime(rolling_avg.Date)
rolling_avg_change = rolling_avg[rolling_avg["Date"] > olddate]
site_avg_change = site_avg[site_avg["Date"] > olddate]

if trouble_flag == 1:
    site_avg_change.tail()

In [34]:
rolling_avg["Date"] = rolling_avg["Date"].dt.to_period("D")
rolling_avg_change["Date"] = rolling_avg_change["Date"].dt.to_period("D")
site_avg["Date"] = site_avg["Date"].dt.to_period("D")
site_avg_change["Date"] = site_avg_change["Date"].dt.to_period("D")

C:\Users\JBC4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\JBC4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [35]:
rolling_avg.to_excel(os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_status\\data\\simplified_site_data.xlsx')
rolling_avg_change.to_excel(os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_status\\data\\site_today.xlsx')
site_avg.to_excel(os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_status\\data\\by_site.xlsx')
site_avg_change.to_excel(os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_status\\data\\by_site_today.xlsx')

## Power Status Update

This portion of the script updates the power status data for the power status board. and consolidates the report by site.

Data Source:  RPS (HOO report) because the data warehouse doesn't update power status until later in the day and people are generally interested in this data early in the morning.


In [37]:
# Download Today's Power Status Report

# Clear  old downloaded .csv file if it exists:
path_file = os.path.expanduser("~")+'\\Downloads\\Power Reactor Status.csv'
if os.path.exists(path_file):
    os.unlink(path_file)

# Get Current Power Status Report

#Set chromedriver path and options:
chrome_path = 'C:\\ProgramData\\Chromedriver\\chromedriver.exe';
chrome_options = Options()
chrome_options.add_argument('--disable-extensions')
chrome_options.add_experimental_option('useAutomationExtension',False)
browser = webdriver.Chrome(chrome_path,options=chrome_options)

# Download the .csv power status report from RRPS:
browser.get('https://hqvwdbrps04.nrc.gov/reportserver/pages/reportviewer.aspx?/Oversight/Reports/Power+Reactor+Status')
time.sleep(4)
browser.execute_script("$find('ReportViewerControl').exportReport('CSV');")
time.sleep(5)
browser.close()

In [38]:
# Read in data files:
power_data = pd.read_csv(path_file)
file_path_site_hist = os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_covid_cumulative_site_history.csv'
file_path_updates = os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_Impacts_for_updating.xlsx'
site_hist_covid = pd.read_csv(file_path_site_hist)
site_info_update = pd.read_excel(file_path_updates, sheet_name='SiteImpacts')
site_info_fuelfac = pd.read_excel(file_path_updates, sheet_name='Fuel Facilities')

In [39]:
#  Append today's data to the site_cumulative_history file:
date_today = datetime.date.today().strftime("%m/%d/%Y")
if date_today in site_hist_covid.columns:
    print('Already there, not writing')
else:
    site_hist_covid[date_today]=site_info_update['Number of Confirmed Cases Onsite'].append(site_info_fuelfac['Number of Confirmed Cases Onsite'],ignore_index=True)
    site_hist_covid.to_csv(file_path_site_hist,index=False)

    

In [40]:
# format and export updated power data for today:
power_data['Comments3'] = power_data['Comments3'].str.replace('\r\n','')
power_data = power_data.fillna('')

# combine unit information from power data file to site level:

for ii in range(0,len(site_info_update)):
    update_string = ''
    down_count = 0
    unit_count = 0
    for jj in range(0,len(power_data)):
            if site_info_update.iloc[ii,0] in power_data.iloc[jj,1]:
                unit_count = unit_count+1
                if power_data.iloc[jj,3] < 10:
                    down_count = down_count+1
                unit_num = re.findall(r'\d+', power_data.iloc[jj,1])
                if len(unit_num) == 0:
                    if power_data.iloc[jj,3] != 100:
                        update_string = update_string + str(power_data.iloc[jj,3]) + '% ' + power_data.iloc[jj,5] + ' ' + power_data.iloc[jj,4]
                else:
                    if power_data.iloc[jj,3] != 100:
                        update_string = update_string + 'Unit ' + unit_num[0] + ': ' + str(power_data.iloc[jj,3]) + '% ' + power_data.iloc[jj,5] + ' ' + power_data.iloc[jj,4]

    site_info_update.iloc[ii,13]=update_string
    if 'Refueling' in site_info_update.iloc[ii,13]:
        site_info_update.iloc[ii,15] = 'Y'
    else:
        site_info_update.iloc[ii,15] = 'N'
    
    if down_count == unit_count:
        site_info_update.iloc[ii,14] = '0 - all units onsite shutdown'
    elif down_count > 0:
        site_info_update.iloc[ii,14] = '1 - one or more units shutdown'
    else:
        site_info_update.iloc[ii,14] = '2 - all units fully operational'
        
        


In [41]:
#  Display results if in troubleshoot mode::
if trouble_flag == 1:
    site_info_update.iloc[:,[0,13,14,15]]

In [42]:
#  Export to file (was used to copy to site_impacts_for_updating since openpyxl resulted in errors)
#  Now only needed for troubleshooting:

if trouble_flag == 1:
    site_info_update.to_excel('Site Impact Updates.xlsx',index=False)

In [43]:
# the following code calculates the reported history at the site from the cumulative history file:
# Note:  Will likely read zero until the tracking process is resumed.
# read in latest US confirmed and death time history data:

covid_site_confirmed = site_hist_covid

# drop all but the last two weeks of data:
covid_site_confirmed.drop(covid_site_confirmed.iloc[:, 1:-15], inplace = True, axis = 1)


# #### Calculate daily and weekly totals:
# 
# The following code calculates the daily, weekly and biweekly totals for confirmed cases and deaths, since the JHU data always appends the current days date as the last column in their dataset.

new_cases_site_daily = covid_site_confirmed[covid_site_confirmed.columns[-1]]-covid_site_confirmed[covid_site_confirmed.columns[-2]]
new_cases_site_weekly = covid_site_confirmed[covid_site_confirmed.columns[-1]]-covid_site_confirmed[covid_site_confirmed.columns[-6]]
new_cases_site_biweekly = covid_site_confirmed[covid_site_confirmed.columns[-1]]-covid_site_confirmed[covid_site_confirmed.columns[-11]]

# Print totals as a check:
if trouble_flag == 1:
    print('Total new site cases in last 24 hours  :  ' + str(new_cases_site_daily.sum()))
    print('Total new site cases in last week      :  ' + str(new_cases_site_weekly.sum()))
    print('Total new site cases in last two weeks :  ' + str(new_cases_site_biweekly.sum()))

#  Append the new data to the dataframes for confirmed cases and deaths:
covid_site_confirmed['New Cases Daily']=new_cases_site_daily
covid_site_confirmed['New Cases Weekly']=new_cases_site_weekly
covid_site_confirmed['New Cases Biweekly']=new_cases_site_biweekly
covid_site_confirmed['Average for Last Week']=covid_site_confirmed['New Cases Weekly'].div(7).round(0)
covid_site_confirmed['Average Past 2 Weeks']=covid_site_confirmed['New Cases Biweekly'].div(14).round(0)

output_df = covid_site_confirmed.iloc[:,[0,16,17,18,19,20]]
# Write the new files to a .csv file that has the calculated fields as the last columns:
file_path_site_stats = os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_status\\data\\current_confirmed_site_data.csv'
output_df.to_csv(file_path_site_stats)

## Findings and Power Status Data Update

This code pulls the DW source data for the findings search tool and updates the xlsx file on Sharepoint

In [45]:
conn = pyodbc.connect('Driver={SQL Server};' 
                      'Server=hqvwepmdb01;' 
                      'Database=NRC_DW;' 
                      'Trusted_Connection=yes;') 
 
cursor = conn.cursor() 
 
sql_query = pd.read_sql_query('SELECT * FROM NRC_DW.dbo.vw_RPS_Inspections',conn) 

Error: ('HY000', '[HY000] [Microsoft][ODBC SQL Server Driver]Cannot generate SSPI context (0) (SQLDriverConnect); [HY000] [Microsoft][ODBC SQL Server Driver]Cannot generate SSPI context (0)')

In [ ]:
findings_file_path = os.path.expanduser("~") + '\\U.S. NRC\\ROPDashboards - Documents\\Databases\\findings_search\\vw_RPS_Inspections.xlsx'
power_file_path = os.path.expanduser("~") + '\\U.S. NRC\\ROPDashboards - Documents\\Databases\\power_status_report\\power_status_data.xlsx'
current_power_file = os.path.expanduser("~") + '\\U.S. NRC\\ROPDashboards - Documents\\Databases\\power_status_report\\power_status_today.xlsx'

In [ ]:
sql_query.to_excel(findings_file_path,index=False)

In [ ]:
conn = pyodbc.connect('Driver={SQL Server};' 
                      'Server=hqvwepmdb01;' 
                      'Database=NRC_Master;' 
                      'Trusted_Connection=yes;') 
 
cursor = conn.cursor() 
 
sql_query = pd.read_sql_query('SELECT * FROM NRC_Master.dbo.rawRPS_ROE_HOOPowerReactorStatus',conn) 

In [ ]:
# filter power history to only the past two years:
d = datetime.datetime.today() - datetime.timedelta(days=730)
sql_query = sql_query[(sql_query['ReportDate'] > d.strftime('%Y-%m-%d'))]




In [ ]:
# export to file:

sql_query.to_excel(power_file_path,index=False)

# write today's information to file

power_data.to_excel(current_power_file,index=False)

In [ ]:
# copy the current power status file to the Power Status directory with today's date as a filename:

report_date_daily = datetime.date.today().strftime("%m-%d-%Y")
source_report_today = os.path.expanduser("~")+'\\Downloads\\Power Reactor Status.csv'
os.rename(source_report_today,os.path.expanduser("~")+'\\Downloads\\' + report_date_daily + '.csv')
dest_report_dir = os.path.expanduser("~") + '\\U.S. NRC\\ROPDashboards - Documents\\Databases\\power_status_report\\daily\\'
shutil.copy(os.path.expanduser("~")+'\\Downloads\\' + report_date_daily + '.csv', dest_report_dir)


In [ ]:
# Update Event Notification Data:
event_notification_file_path = os.path.expanduser("~") + '\\U.S. NRC\\ROPDashboards - Documents\\Databases\\OpE\\event_notification_raw_data.xlsx'
sql_query = pd.read_sql_query('SELECT * FROM NRC_Master.dbo.rawRPS_ROE_HOOEventNotification',conn)
sql_query.to_excel(event_notification_file_path,index=False)



In [ ]:
if trouble_flag == 1:
    sql_query.Text.head()

### Update COVID board information from CDC and COVID Act Now

In [ ]:
covid_act_now = pd.read_csv('https://api.covidactnow.org/v2/counties.csv?apiKey=84ae403fda674c3d8590a7410f805f47')
covid_act_now.to_csv(os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_status\\data\\covid_act_now.csv',index=False)

In [ ]:
a = requests.get('https://healthdata.gov/api/odata/v4/6hii-ae4f')

In [ ]:
a_text = a.text

In [ ]:
cdc_date_today = datetime.date.today()
cdc_week_ago = cdc_date_today-datetime.timedelta(days=7)
start_date = cdc_week_ago.strftime("%Y%m%d")

In [ ]:
last_date = [] 
for ii in range(1,7):
        check_date = cdc_date_today-datetime.timedelta(days=ii)
        search_string = check_date.strftime("%Y%m%d") + '.xlsx'
        if search_string in a_text:
            last_date = search_string
            break

In [ ]:
loc_id = a_text.find(search_string)

In [ ]:
hash_file_id = a_text[loc_id+32:loc_id+68]

In [ ]:
hash_file_id

In [ ]:
cdc_file_url = 'https://healthdata.gov/api/views/gqxm-d9w9/files/'+hash_file_id+'?download=true&filename=Community%20Profile%20Report%'+search_string


In [ ]:
import urllib
outfilename = os.path.expanduser("~") + '\\U.S. NRC\\COVID-19 Site Impacts - Documents\\site_status\\data\\Community_Profile_Report.xlsx'
urllib.request.urlretrieve(cdc_file_url, outfilename) 

In [ ]:
last_date

### Update OpE Documents Master

In [ ]:

LER_url = 'https://adams.nrc.gov/wba/services/search/advanced/nrc?q=(mode:sections,sections:(filters:(public-library:!t),options:(within-folder:(enable:!f,insubfolder:!f,path:%27%27)),properties_search_all:!(!(DocumentType,starts,%27Licensee+Event+Report%27,%27%27),!(DocketNumber,starts,%2705000%27,%27%27),!(DocumentDate,range,(left:%2701/01/2021+12:00+AM%27,right:%2712/31/2021+11:59+PM%27),%27%27))))&qn=New&tab=advanced-search-pars&s=%24title&so=ASC'
  
# creating HTTP response object from given url
resp = requests.get(LER_url)
# saving the xml file
with open('LERfeed.xml', 'wb') as f:
    f.write(resp.content)

      
xml_data = open('LERfeed.xml', 'r').read()  # Read file
root = ET.XML(xml_data)  # Parse XML

In [ ]:
LER_titles = []
LER_MLs = []
LER_date = []
LER_doctype = []
LER_link = []
for child in root.iter():
    if child.tag == 'AccessionNumber':
        LER_MLs.append(child.text)
        LER_link.append('https://www.nrc.gov/docs/'+ child.text[0:6] + '/' + child.text + '.pdf')
    elif child.tag == 'DocumentTitle':
        LER_titles.append(child.text)
    elif child.tag == 'DocumentDate':
        LER_date.append(child.text)
    elif child.tag == 'DocumentType':
        LER_doctype.append(child.text)



In [ ]:
document_columns = ['Title', 'AccessionNumber','DocumentDate','DocumentType','Link']
LER_data = pd.DataFrame(list(zip(LER_titles,LER_MLs,LER_date,LER_doctype,LER_link)),columns = document_columns )

In [ ]:
LER_data.head()

In [ ]:

def ADAMS_pull(document_type, year):
    doc_str = document_type.replace(' ','+')
    API_url = 'https://adams.nrc.gov/wba/services/search/advanced/nrc?q=(mode:sections,sections:(filters:(public-library:!t),options:(within-folder:(enable:!f,insubfolder:!f,path:%27%27)),properties_search_all:!(!(DocumentType,starts,%27' + doc_str + '%27,%27%27),!(DocketNumber,starts,%2705000%27,%27%27),!(DocumentDate,range,(left:%2701/01/' + str(year) +'+12:00+AM%27,right:%2712/31/' + str(year) + '+11:59+PM%27),%27%27))))&qn=New&tab=advanced-search-pars&s=%24title&so=ASC'
    # creating HTTP response object from given url
    resp = requests.get(API_url)
    
    # saving the xml file
    with open('document_feed.xml', 'wb') as f:
        f.write(resp.content)

    xml_data = open('document_feed.xml', 'r').read()  # Read file
    root = ET.XML(xml_data)  # Parse XML
    
    doc_titles = []
    doc_MLs = []
    doc_date = []
    doc_doctype = []
    doc_link = []
    for child in root.iter():
        if child.tag == 'AccessionNumber':
            doc_MLs.append(child.text)
            doc_link.append('https://www.nrc.gov/docs/'+ child.text[0:6] + '/' + child.text + '.pdf')
        elif child.tag == 'DocumentTitle':
            doc_titles.append(child.text)
        elif child.tag == 'DocumentDate':
            doc_date.append(child.text)
        elif child.tag == 'DocumentType':
            doc_doctype.append(child.text)
    
    document_columns = ['Title', 'AccessionNumber','DocumentDate','DocumentType','Link']
    doc_data = pd.DataFrame(list(zip(doc_titles,doc_MLs,doc_date,doc_doctype,doc_link)),columns = document_columns )
    
    return doc_data

In [ ]:
LER_2020 = ADAMS_pull('Deficiency',2020)

In [ ]:
LER_2020